### !필수체크!
* decode_env 를 잡고 있는지 확인
* GPU 확인
* Dataset PATH 확인

In [ ]:
import os, torch

print("현재 접속한 파이썬 가상환경은 {} 입니다. (decode_env 인지 확인할 것)\n".format(os.environ['CONDA_DEFAULT_ENV']))

print("사용 가능한 GPU가 존재하는가? (True or False): ", torch.cuda.is_available())
if torch.cuda.is_available():
    print("사용 가능한 GPU의 수는 {} 개 입니다.".format(torch.cuda.device_count()))
    print("GPU 각각의 이름은 아래와 같습니다.")
    for i in range(torch.cuda.device_count()):
        print("GPU {}: {}".format(i, torch.cuda.get_device_name(i)))
else:
    print("사용 가능한 GPU가 존재하지 않습니다. 혹은 GPU를 Pytorch가 찾지 못하고 있습니다.")

data_path = '/home/bbl/Dataset' # Dataset 폴더 지정

In [ ]:
import decode
import torch
import matplotlib.pyplot as plt

print(f"DECODE version: {decode.utils.bookkeeping.decode_state()}")

# DECODE - Evaluation
In the following we show what DECODE provides in terms of evaluation.
Please be advised to have a read of the Introduction notebook first.
Evaluation always acts on two sets of emitters, mostly prediction and reference (i.e. ground truth). In the following we show some toy examples.

## Evaluation Pipeline
The evaluation workflow is almost always
1. Definition of reference and candidate (=prediction) EmitterSet
2. Matching of reference and candidate
3. Evaluation of Detection and Localization Metrics

In [ ]:
# dummy emittersets
em_ref = decode.RandomEmitterSet(200, xy_unit='px', px_size=(100., 100.))
em_ref.frame_ix = torch.randint_like(em_ref.frame_ix, low=0, high=20)

em_pred = em_ref.clone()  # make independent copy
em_pred = em_pred[torch.rand(len(em_pred)) <= 0.8]  # through away random subset of 20%
em_pred.xyz += torch.randn_like(em_pred.xyz)  # wiggle coordinates a bit around

In [ ]:
# plot all emitters
decode.plot.PlotCoordinates(pos_tar=em_ref.xyz_px, 
                            pos_out=em_pred.xyz_px).plot()
plt.legend()
plt.title('All Emitters')
plt.show()

# plot on a selected frame
decode.plot.PlotCoordinates(pos_tar=em_ref[em_ref.frame_ix == 10].xyz_px, 
                            pos_out=em_pred[em_pred.frame_ix == 10].xyz_px).plot()
plt.legend()
plt.title('Single Frame')
plt.show()

## Matching
We perform matching like in the superres fight club (THE challenge). This means that one defines a search radius in which predictions are considered true positives. We have 1:1 matches.

In [ ]:
matcher = decode.evaluation.match_emittersets.GreedyHungarianMatching(match_dims=3, dist_lat=250, dist_ax=500.)  # always in nm

tp, fp, fn, tp_match = matcher.forward(em_pred, em_ref)  # outputs true positives, false positives, false negatives, matched ground truth

In [ ]:
# plot
# plot on a selected frame
decode.plot.PlotCoordinates(pos_tar=tp_match[tp_match.frame_ix == 10].xyz_px, 
                            pos_out=tp[tp.frame_ix == 10].xyz_px,
                            pos_ini=em_ref[em_ref.frame_ix == 10].xyz_px,
                            match_lines=True).plot()

plt.legend()
plt.title('Single Frame with matches connected by the lines')
plt.show()

## Performance Evaluation
Below we perform the actual evaluation which is found in the convenience class `SMLMEvaluation`. Note that some of the evaluation metrices, i.e. the CRLB based ones, require special inputs which can not be computed from two emittersets alone. In Order to evaluate also those, you need to specify a PSF function, provide background estimates and calculate the CRLB:

    emitter.populate_crlb(psf)
    
If these values are not present, a warning will be issued (`UserWarning: Non-Finite values encountered during fitting.`), it can be safely ignored though.

In [ ]:
evaluator = decode.evaluation.SMLMEvaluation()

result = evaluator.forward(tp, fp, fn, tp_match)

In [ ]:
# performance values
result._asdict()

In [ ]:
# more readable description of the metrices.
evaluator.descriptors